In [1]:
import torch 
import torchvision 
import numpy as np

import os
import pandas as pd
import albumentations as A
import albumentations.pytorch
import cv2

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
# import torch.nn.functional as F

import sys
sys.path.append('../')
from dataset import val_transforms, CDataset

In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 64, 
                          "epochs": 40, 
                          "data": 0, 
                          'lr':0.02,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':16,
                         'print_freq':2000,
                         'saved_dir':'../trained_models/checkpoint.pt'})

In [3]:
# make saved dir
from pathlib import Path
path = Path(args.saved_dir.split('checkpoint')[0])
path.mkdir(parents=True, exist_ok=True)

In [4]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

3
cuda:0


In [5]:
from augment import *

In [6]:
IMAGE_SIZE = 256
train_transforms = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1),
    A.OneOf([
        A.HorizontalFlip(p=.8),
        A.VerticalFlip(p=.8),
        A.RandomRotate90(p=.8)]
    ),
    # A.OneOf([
    #     HEColorAugment(sigma1=.2, sigma2=2., mat=None, p=0.9),
    #     HEColorAugment(sigma1=.4, sigma2=5., mat=None, p=0.9),
    #     HEColorAugment(sigma1=.6, sigma2=5., mat=None, p=0.9)
    # ]),
], p=1.0) 

In [7]:
import albumentations as A
import albumentations.pytorch
import cv2

class CDataset(Dataset):
    def __init__(self, df, defaultpath='/home/beomgon/Dataset/new_patches/', transform=None):
        self.df = df
        self.transform = transform
        self.dir = defaultpath

    def __len__(self):
        return len(self.df)   

    def __getitem__(self, idx):
        path = self.df.iloc[idx, 4]
#         print(pid)

        image = cv2.imread(self.dir + path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = (image.astype(np.float32)-128.)/128.
        
#         if image is uint8, normalization by 255 is done automatically by albumebtation(ToTensor method)
        if self.transform:
            timage = self.transform(image=image)
            image = timage['image']
        
        image =  torch.tensor(image, dtype=torch.float32)/255.
        #image = (torch.tensor(image, dtype=torch.float32)-128)/128
        image = image.permute(2,0,1)
            
        label = self.df.iloc[idx, 5]
        return image, label, path

In [8]:
train_df = pd.read_csv('../dataframe/train_New_Stomach_df.csv')
train_dataset = CDataset(train_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/', 
                         transform=train_transforms)

val_df = pd.read_csv('../dataframe/val_New_Stomach_df.csv')
val_dataset = CDataset(val_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/',
                       transform=val_transforms)  

test_df = pd.read_csv('../dataframe/test_New_Stomach_df.csv')
test_dataset = CDataset(test_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/',
                        transform=val_transforms)  

In [9]:
image, label, path = next(iter(train_dataset))
# image

In [10]:
train_loader = DataLoader(train_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

val_loader = DataLoader(val_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

test_loader = DataLoader(test_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)


In [11]:
# images, labels, paths = next(iter(train_loader))
# images.shape

In [12]:
# import torchvision.models as models
from resnet import resnet152, resnet18
model = resnet18(pretrained=False)
model.fc = nn.Linear(16, 3)
torch.nn.init.normal_(model.fc.weight, std=0.01)
torch.nn.init.constant_(model.fc.bias, 0.)
default_state_dict = model.state_dict()
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.01, weight_decay=args.weight_decay)
optimizer = torch.optim.SGD(
       params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = torch.optim.SGD(params, lr=args.lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30], 
                                                    gamma=0.2)
# criterion = nn.CrossEntropyLoss(label_smoothing=0.0)
criterion = nn.CrossEntropyLoss().to(device)

In [16]:
from main import *

model = resnet18(pretrained=False, linear_evaluation=True)
model.fc = nn.Linear(16, 3)
args.saved_dir = '../trained_models/resnet8/cus_stomach4_' + str(1) + '/checkpoint.pt'
checkpoint = torch.load(args.saved_dir)
model.load_state_dict(checkpoint['state_dict'])
model.train()
for p in model.parameters() :
    p.requires_grad = False
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True
model.to(device)

best_acc1 = 0
acc1 = 0

optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=args.weight_decay)

# acc1 = validate(val_loader, model, criterion, args)   
# print('acc1', acc1)

for epoch in range(args.epochs) :
    model.train()
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30], 
                                                        gamma=0.3)    
    scheduler.step()
    for i, (images, targets, _) in enumerate(train_loader):

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            targets = targets.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, targets)    
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
    # evaluate after every epoch
    acc1 = validate(val_loader, model, criterion, args)   

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1) 

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
        'scheduler' : scheduler.state_dict(),
    }, is_best, filename=args.saved_dir)           

Test: [  0/406]	Time  1.384 ( 1.384)	Loss 3.6756e-01 (3.6756e-01)	Acc@1  87.50 ( 87.50)
 * Acc@1 90.493
Test: [  0/406]	Time  1.374 ( 1.374)	Loss 2.8977e-01 (2.8977e-01)	Acc@1  92.19 ( 92.19)
 * Acc@1 89.900
Test: [  0/406]	Time  1.321 ( 1.321)	Loss 2.7304e-01 (2.7304e-01)	Acc@1  92.19 ( 92.19)
 * Acc@1 90.713
Test: [  0/406]	Time  1.159 ( 1.159)	Loss 1.9268e-01 (1.9268e-01)	Acc@1  92.19 ( 92.19)
 * Acc@1 90.605
Test: [  0/406]	Time  1.302 ( 1.302)	Loss 2.4541e-01 (2.4541e-01)	Acc@1  93.75 ( 93.75)
 * Acc@1 90.671
Test: [  0/406]	Time  1.198 ( 1.198)	Loss 5.3538e-01 (5.3538e-01)	Acc@1  85.94 ( 85.94)
 * Acc@1 90.621
Test: [  0/406]	Time  1.174 ( 1.174)	Loss 2.9040e-01 (2.9040e-01)	Acc@1  89.06 ( 89.06)
 * Acc@1 90.559
Test: [  0/406]	Time  1.168 ( 1.168)	Loss 3.3278e-01 (3.3278e-01)	Acc@1  90.62 ( 90.62)
 * Acc@1 90.786
Test: [  0/406]	Time  1.364 ( 1.364)	Loss 4.1314e-01 (4.1314e-01)	Acc@1  85.94 ( 85.94)
 * Acc@1 90.362
Test: [  0/406]	Time  1.068 ( 1.068)	Loss 3.0331e-01 (3.0331e-01

In [17]:
args.saved_dir = '../trained_models/resnet8/cus_stomach4_' + str(1) + '/checkpoint.pt'
checkpoint = torch.load(args.saved_dir)
model.load_state_dict(checkpoint['state_dict'])
acc1 = validate(test_loader, model, criterion, args)    

Test: [  0/237]	Time  1.254 ( 1.254)	Loss 1.2496e-01 (1.2496e-01)	Acc@1  93.75 ( 93.75)
 * Acc@1 92.883


In [18]:
acc1 = validate(val_loader, model, criterion, args)    

Test: [  0/406]	Time  1.386 ( 1.386)	Loss 4.7145e-01 (4.7145e-01)	Acc@1  85.94 ( 85.94)
 * Acc@1 90.863
